In [1]:
import graphlab as gl

In [18]:
def _user_exists(rating_data, user_id):
    if not rating_data:
        return False
    return user_id in rating_data['user_id']

class Recommender:
    def __init__(self, user_rating_filepath, restaurant_cuisine_filepath):
        self.rating_data = gl.SFrame.read_csv(user_rating_filepath, column_type_hints={"rating":int})
        self.restaurant_cuisine_data = gl.SFrame.read_csv(restaurant_cuisine_filepath)
        self.new_rating_data = {}
        self.restaurant_recommender = gl.item_similarity_recommender.create(
            self.rating_data,
            target="rating",
            verbose=False)
        self.popularity_recommender = gl.popularity_recommender.create(
            self.rating_data,
            target="rating",
            verbose=False)
        
    def recommend(self, user_id, query=None):
        recommender = self.restaurant_recommender # if self.__is_not_new_user(user_id) else self.popularity_recommender
        user_rating_data = self.new_rating_data.get(user_id)
        top_items = recommender.recommend(
            users=[user_id],
            k=5,
            items=self.__filter_restaurants(query) if query else None,
            new_observation_data=user_rating_data,
            exclude_known=False if query else True,
            verbose=False)
        return top_items, user_rating_data
    
    def add_rating(self, user_id, restaurant_id, rating):
        row = gl.SFrame({
                'user_id': [user_id],
                'item_id': [restaurant_id],
                'rating': [rating]
        })
        user_rating_data = self.new_rating_data.get(user_id)
        if user_rating_data:
            user_rating_data = user_rating_data.append(row)
        else:
            user_rating_data = row
        self.new_rating_data[user_id] = user_rating_data
        
    def __filter_restaurants(self, query):
        query = query.lower().encode('utf-8')
        cuisine_filter = self.restaurant_cuisine_data['cuisine'].apply(lambda x: query in x.lower())
        return self.restaurant_cuisine_data[cuisine_filter]['item_id']
    
    def __is_not_new_user(self, user_id):
        return _user_exists(self.new_rating_data, user_id)

In [19]:
recommendation_engine = Recommender("data/user_restaurant_rating.csv", "data/restaurant_cuisine.csv")

Finished parsing file /Users/danielconger/Desktop/thesis-python-ml/data/user_restaurant_rating.csv

Parsing completed. Parsed 100 lines in 0.277742 secs.

Finished parsing file /Users/danielconger/Desktop/thesis-python-ml/data/user_restaurant_rating.csv

Parsing completed. Parsed 103428 lines in 0.17841 secs.

Finished parsing file /Users/danielconger/Desktop/thesis-python-ml/data/restaurant_cuisine.csv

...

Finished parsing file /Users/danielconger/Desktop/thesis-python-ml/data/restaurant_cuisine.csv

Parsing completed. Parsed 4366 lines in 0.019631 secs.

Recsys training: model = item_similarity

Recsys training: model = popularity

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [20]:
top_items, user_rating_data  = recommendation_engine.recommend('ZY1qKEWM8mUYTpyQYXfxKg', 'italian')
top_items

user_id,item_id,score,rank
ZY1qKEWM8mUYTpyQYXfxKg,s5-VDvUqyYnD8RG3LPqnbQ,0.0121893400238,1
ZY1qKEWM8mUYTpyQYXfxKg,mgN7A6Uw8ObmoTNZvx6VOg,0.0116578662206,2
ZY1qKEWM8mUYTpyQYXfxKg,6X01fXVzhCght4ZJ9Ei2JQ,0.0105137768246,3
ZY1qKEWM8mUYTpyQYXfxKg,i3ogCoQD4MY_sR73GCk-BA,0.0102148292557,4
ZY1qKEWM8mUYTpyQYXfxKg,3MIy-ca6QVtKWXNzerQxWw,0.0101740852235,5


In [14]:
recommendation_engine.add_rating('ZY1qKEWM8mUYTpyQYXfxKg', '8buIr1zBCO7OEcAQSZko7w', 4)

In [16]:
user_rating_data

item_id,rating,user_id
8buIr1zBCO7OEcAQSZko7w,4,ZY1qKEWM8mUYTpyQYXfxKg


In [17]:
recommendation_engine.new_rating_data

{'ZY1qKEWM8mUYTpyQYXfxKg': Columns:
 	item_id	str
 	rating	int
 	user_id	str
 
 Rows: 1
 
 Data:
 +------------------------+--------+------------------------+
 |        item_id         | rating |        user_id         |
 +------------------------+--------+------------------------+
 | 8buIr1zBCO7OEcAQSZko7w |   4    | ZY1qKEWM8mUYTpyQYXfxKg |
 +------------------------+--------+------------------------+
 [1 rows x 3 columns]}